In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords
from collections import Counter
from string import digits
from tqdm import tqdm
import spacy
from nltk.stem import SnowballStemmer
import warnings
from langdetect import detect
from langdetect import DetectorFactory   

warnings.filterwarnings("ignore")

# data preprocessing

In [2]:
df = pd.read_csv("extracted_data.csv")

In [3]:
df

,Nachrichtentyp,Datun,Uhrzeit,Absender,Ort,Text,ID
0,Beschreibung,2015.11.25,19:35:19,H2451TK1,<LOCATION>,Hallo zusammen\n\naktuell sind die Systeme i06...,2000000060
1,Beschreibung,2015.11.26,14:49:28,H2741CR1,<LOCATION>,"Ich weiß nicht, woher ich die SW Information D...",2000000070
2,Beschreibung,2015.11.26,15:37:47,H2741CR1,<LOCATION>,Da ich keine neue Connection anlegen kann (ist...,2000000071
3,Beschreibung,2015.11.30,11:14:57,H2702AR1,<LOCATION>,"Guten Morgen, SAP UCC Team,\n\nich würde gerne...",2000000080
4,Beschreibung,2015.12.01,10:06:57,H2941JK1,<LOCATION>,"Wie bereits früher in Mails dokumentiert, läuf...",2000000081
...,...,...,...,...,...,...,...
6330,Beschreibung,2021.06.02,11:04:00,U2PL010SS1,<LOCATION>,"Good morning,\nplease generate dev keys. for U...",2000012395
6331,Beschreibung,2021.06.02,16:44:10,H2901RW1,<LOCATION>,"Sehr geehrtes SAP UCC Support Team,\n\nin eine...",2000012396
6332,Beschreibung,2021.06.03,07:38:44,H2781JW2,<LOCATION>,"Hallo zusammen,\n\nkönnen Sie bitte das Passwo...",2000012397
6333,Beschreibung,2021.06.03,12:25:44,U2ES024CO1,<LOCATION>,"My colleague, <PERSON2113> (<PERSON2112>), can...",2000012398


In [4]:
# Remove sensitive words
df["text_wo_seinfo"] = df.Text.replace(r'<PERSON[0-9_]*>', '', regex=True)
df["text_wo_seinfo"] = df.text_wo_seinfo.replace(
    r'<LOCATION[0-9_]*>', '', regex=True)

In [5]:
# Remove the carriage return, tab, line feed
df["text_wo_seinfo_wo_enter"] = df.text_wo_seinfo.replace(
    r'\r+|\n+|\t+', ' ', regex=True)

In [6]:
# Lowercase words
df["text_wo_seinfo_wo_enter_lower"] = df["text_wo_seinfo_wo_enter"].str.lower()

In [7]:
 print("there are the punctuations that will be  converted to spaces: "+string.punctuation)
# symbols are converted to spaces
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text


df["text_wo_seinfo_wo_enter_lower_wo_punct"] = df["text_wo_seinfo_wo_enter_lower"].apply(
    lambda text: remove_punctuations(text))
df.head()

there are the punctuations that will be  converted to spaces: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


,Nachrichtentyp,Datun,Uhrzeit,Absender,Ort,Text,ID,text_wo_seinfo,text_wo_seinfo_wo_enter,text_wo_seinfo_wo_enter_lower,text_wo_seinfo_wo_enter_lower_wo_punct
0,Beschreibung,2015.11.25,19:35:19,H2451TK1,<LOCATION>,Hallo zusammen\n\naktuell sind die Systeme i06...,2000000060,Hallo zusammen\n\naktuell sind die Systeme i06...,"Hallo zusammen aktuell sind die Systeme i06, i...","hallo zusammen aktuell sind die systeme i06, i...",hallo zusammen aktuell sind die systeme i06 i...
1,Beschreibung,2015.11.26,14:49:28,H2741CR1,<LOCATION>,"Ich weiß nicht, woher ich die SW Information D...",2000000070,"Ich weiß nicht, woher ich die SW Information D...","Ich weiß nicht, woher ich die SW Information D...","ich weiß nicht, woher ich die sw information d...",ich weiß nicht woher ich die sw information d...
2,Beschreibung,2015.11.26,15:37:47,H2741CR1,<LOCATION>,Da ich keine neue Connection anlegen kann (ist...,2000000071,Da ich keine neue Connection anlegen kann (ist...,Da ich keine neue Connection anlegen kann (ist...,da ich keine neue connection anlegen kann (ist...,da ich keine neue connection anlegen kann ist...
3,Beschreibung,2015.11.30,11:14:57,H2702AR1,<LOCATION>,"Guten Morgen, SAP UCC Team,\n\nich würde gerne...",2000000080,"Guten Morgen, SAP UCC Team,\n\nich würde gerne...","Guten Morgen, SAP UCC Team, ich würde gerne di...","guten morgen, sap ucc team, ich würde gerne di...",guten morgen sap ucc team ich würde gerne di...
4,Beschreibung,2015.12.01,10:06:57,H2941JK1,<LOCATION>,"Wie bereits früher in Mails dokumentiert, läuf...",2000000081,"Wie bereits früher in Mails dokumentiert, läuf...","Wie bereits früher in Mails dokumentiert, läuf...","wie bereits früher in mails dokumentiert, läuf...",wie bereits früher in mails dokumentiert läuf...


In [8]:
# Stopwords that will be processed
", ".join(stopwords.words('english')+stopwords.words('german'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [9]:
# Remove stopwords
STOPWORDS = set(stopwords.words('english')+stopwords.words('german'))


def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])


df["text_wo_seinfo_wo_enter_lower_wo_punct_wo_stop"] = df["text_wo_seinfo_wo_enter_lower_wo_punct"].apply(
    lambda text: remove_stopwords(text))
df.head()

,Nachrichtentyp,Datun,Uhrzeit,Absender,Ort,Text,ID,text_wo_seinfo,text_wo_seinfo_wo_enter,text_wo_seinfo_wo_enter_lower,text_wo_seinfo_wo_enter_lower_wo_punct,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stop
0,Beschreibung,2015.11.25,19:35:19,H2451TK1,<LOCATION>,Hallo zusammen\n\naktuell sind die Systeme i06...,2000000060,Hallo zusammen\n\naktuell sind die Systeme i06...,"Hallo zusammen aktuell sind die Systeme i06, i...","hallo zusammen aktuell sind die systeme i06, i...",hallo zusammen aktuell sind die systeme i06 i...,hallo zusammen aktuell systeme i06 i58 i72 err...
1,Beschreibung,2015.11.26,14:49:28,H2741CR1,<LOCATION>,"Ich weiß nicht, woher ich die SW Information D...",2000000070,"Ich weiß nicht, woher ich die SW Information D...","Ich weiß nicht, woher ich die SW Information D...","ich weiß nicht, woher ich die sw information d...",ich weiß nicht woher ich die sw information d...,weiß woher sw information design tool bekomme ...
2,Beschreibung,2015.11.26,15:37:47,H2741CR1,<LOCATION>,Da ich keine neue Connection anlegen kann (ist...,2000000071,Da ich keine neue Connection anlegen kann (ist...,Da ich keine neue Connection anlegen kann (ist...,da ich keine neue connection anlegen kann (ist...,da ich keine neue connection anlegen kann ist...,neue connection anlegen ausgegraut nehme beste...
3,Beschreibung,2015.11.30,11:14:57,H2702AR1,<LOCATION>,"Guten Morgen, SAP UCC Team,\n\nich würde gerne...",2000000080,"Guten Morgen, SAP UCC Team,\n\nich würde gerne...","Guten Morgen, SAP UCC Team, ich würde gerne di...","guten morgen, sap ucc team, ich würde gerne di...",guten morgen sap ucc team ich würde gerne di...,guten morgen sap ucc team gerne kursmaterialie...
4,Beschreibung,2015.12.01,10:06:57,H2941JK1,<LOCATION>,"Wie bereits früher in Mails dokumentiert, läuf...",2000000081,"Wie bereits früher in Mails dokumentiert, läuf...","Wie bereits früher in Mails dokumentiert, läuf...","wie bereits früher in mails dokumentiert, läuf...",wie bereits früher in mails dokumentiert läuf...,bereits früher mails dokumentiert läuft hinter...


In [10]:
# Counting high frequency words
cnt = Counter()
for text in df["text_wo_seinfo_wo_enter_lower_wo_punct_wo_stop"].values:
    for word in text.split():
        cnt[word] += 1

In [11]:
# The 130 most frequent words
cnt.most_common(130)

[('sap', 4112),
 ('system', 3508),
 ('please', 1868),
 ('bitte', 1672),
 ('gbi', 1559),
 ('grüße', 1516),
 ('name', 1511),
 ('client', 1498),
 ('team', 1449),
 ('hana', 1385),
 ('user', 1341),
 ('3', 1313),
 ('dank', 1289),
 ('regards', 1263),
 ('domain', 1262),
 ('hallo', 1254),
 ('viele', 1208),
 ('ucc', 1162),
 ('2', 1160),
 ('1', 1058),
 ('vielen', 1026),
 ('mandanten', 959),
 ('geehrte', 954),
 ('herren', 945),
 ('damen', 942),
 ('grüßen', 914),
 ('freundlichen', 891),
 ('problem', 801),
 ('could', 765),
 ('university', 763),
 ('email', 742),
 ('4', 737),
 ('new', 713),
 ('hi', 699),
 ('best', 696),
 ('students', 693),
 ('mandant', 690),
 ('would', 687),
 ('version', 684),
 ('thanks', 678),
 ('gui', 661),
 ('error', 642),
 ('access', 638),
 ('support', 631),
 ('0', 614),
 ('data', 591),
 ('leider', 584),
 ('dear', 578),
 ('address', 576),
 ('ip', 539),
 ('us', 521),
 ('thank', 515),
 ('need', 507),
 ('danke', 499),
 ('server', 497),
 ('master', 496),
 ('liebes', 474),
 ('10', 472)

In [12]:
# Keep the meaningful part of the 130 high-frequency words
FREQWORDS = set([w for (w, wc) in cnt.most_common(130)])
FREQWORDS.difference_update(["client", "hana", "gbi", "user", "team", "mandanten", "university", "students", "mandant", "version", "access", "connection", "domain", "mail"
                             "error", "gui", "data", "fehlermeldung", "reset", "password", "erpsim", "fehler", "service", "master", "erp", "email", "server", "id", "ides"
                             "ip", "hochschule", "student", "request", "login", "users", "contract", "portal", "message", "case", "tum", "4hana", "remote", "ticket", "lumira"])


# Remove the words that have no obvious meaning among the 130 high-frequency words
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])


df["text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq"] = df["text_wo_seinfo_wo_enter_lower_wo_punct_wo_stop"].apply(
    lambda text: remove_freqwords(text))
df.head()

,Nachrichtentyp,Datun,Uhrzeit,Absender,Ort,Text,ID,text_wo_seinfo,text_wo_seinfo_wo_enter,text_wo_seinfo_wo_enter_lower,text_wo_seinfo_wo_enter_lower_wo_punct,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stop,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq
0,Beschreibung,2015.11.25,19:35:19,H2451TK1,<LOCATION>,Hallo zusammen\n\naktuell sind die Systeme i06...,2000000060,Hallo zusammen\n\naktuell sind die Systeme i06...,"Hallo zusammen aktuell sind die Systeme i06, i...","hallo zusammen aktuell sind die systeme i06, i...",hallo zusammen aktuell sind die systeme i06 i...,hallo zusammen aktuell systeme i06 i58 i72 err...,aktuell systeme i06 i58 i72 erreichbar fehlerm...
1,Beschreibung,2015.11.26,14:49:28,H2741CR1,<LOCATION>,"Ich weiß nicht, woher ich die SW Information D...",2000000070,"Ich weiß nicht, woher ich die SW Information D...","Ich weiß nicht, woher ich die SW Information D...","ich weiß nicht, woher ich die sw information d...",ich weiß nicht woher ich die sw information d...,weiß woher sw information design tool bekomme ...,weiß woher sw design tool bekomme daher auskun...
2,Beschreibung,2015.11.26,15:37:47,H2741CR1,<LOCATION>,Da ich keine neue Connection anlegen kann (ist...,2000000071,Da ich keine neue Connection anlegen kann (ist...,Da ich keine neue Connection anlegen kann (ist...,da ich keine neue connection anlegen kann (ist...,da ich keine neue connection anlegen kann ist...,neue connection anlegen ausgegraut nehme beste...,neue connection anlegen ausgegraut nehme beste...
3,Beschreibung,2015.11.30,11:14:57,H2702AR1,<LOCATION>,"Guten Morgen, SAP UCC Team,\n\nich würde gerne...",2000000080,"Guten Morgen, SAP UCC Team,\n\nich würde gerne...","Guten Morgen, SAP UCC Team, ich würde gerne di...","guten morgen, sap ucc team, ich würde gerne di...",guten morgen sap ucc team ich würde gerne di...,guten morgen sap ucc team gerne kursmaterialie...,guten morgen team kursmaterialien erp configur...
4,Beschreibung,2015.12.01,10:06:57,H2941JK1,<LOCATION>,"Wie bereits früher in Mails dokumentiert, läuf...",2000000081,"Wie bereits früher in Mails dokumentiert, läuf...","Wie bereits früher in Mails dokumentiert, läuf...","wie bereits früher in mails dokumentiert, läuf...",wie bereits früher in mails dokumentiert läuf...,bereits früher mails dokumentiert läuft hinter...,früher mails dokumentiert läuft hintergrund an...


In [13]:
# Remove rare words with count less than 3
RAREWORDS = set([w for (w, wc) in cnt.most_common() if wc < 3])
print("rare words with count less than 3 are :"+str(len(RAREWORDS)))


def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])


df["text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare"] = df[
    "text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
df.head()

rare words with count less than 3 are :15185


,Nachrichtentyp,Datun,Uhrzeit,Absender,Ort,Text,ID,text_wo_seinfo,text_wo_seinfo_wo_enter,text_wo_seinfo_wo_enter_lower,text_wo_seinfo_wo_enter_lower_wo_punct,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stop,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare
0,Beschreibung,2015.11.25,19:35:19,H2451TK1,<LOCATION>,Hallo zusammen\n\naktuell sind die Systeme i06...,2000000060,Hallo zusammen\n\naktuell sind die Systeme i06...,"Hallo zusammen aktuell sind die Systeme i06, i...","hallo zusammen aktuell sind die systeme i06, i...",hallo zusammen aktuell sind die systeme i06 i...,hallo zusammen aktuell systeme i06 i58 i72 err...,aktuell systeme i06 i58 i72 erreichbar fehlerm...,aktuell systeme i06 i58 i72 erreichbar fehlerm...
1,Beschreibung,2015.11.26,14:49:28,H2741CR1,<LOCATION>,"Ich weiß nicht, woher ich die SW Information D...",2000000070,"Ich weiß nicht, woher ich die SW Information D...","Ich weiß nicht, woher ich die SW Information D...","ich weiß nicht, woher ich die sw information d...",ich weiß nicht woher ich die sw information d...,weiß woher sw information design tool bekomme ...,weiß woher sw design tool bekomme daher auskun...,weiß woher sw design tool bekomme daher auskun...
2,Beschreibung,2015.11.26,15:37:47,H2741CR1,<LOCATION>,Da ich keine neue Connection anlegen kann (ist...,2000000071,Da ich keine neue Connection anlegen kann (ist...,Da ich keine neue Connection anlegen kann (ist...,da ich keine neue connection anlegen kann (ist...,da ich keine neue connection anlegen kann ist...,neue connection anlegen ausgegraut nehme beste...,neue connection anlegen ausgegraut nehme beste...,neue connection anlegen ausgegraut nehme beste...
3,Beschreibung,2015.11.30,11:14:57,H2702AR1,<LOCATION>,"Guten Morgen, SAP UCC Team,\n\nich würde gerne...",2000000080,"Guten Morgen, SAP UCC Team,\n\nich würde gerne...","Guten Morgen, SAP UCC Team, ich würde gerne di...","guten morgen, sap ucc team, ich würde gerne di...",guten morgen sap ucc team ich würde gerne di...,guten morgen sap ucc team gerne kursmaterialie...,guten morgen team kursmaterialien erp configur...,guten morgen team erp configuration gbi nutzen...
4,Beschreibung,2015.12.01,10:06:57,H2941JK1,<LOCATION>,"Wie bereits früher in Mails dokumentiert, läuf...",2000000081,"Wie bereits früher in Mails dokumentiert, läuf...","Wie bereits früher in Mails dokumentiert, läuf...","wie bereits früher in mails dokumentiert, läuf...",wie bereits früher in mails dokumentiert läuf...,bereits früher mails dokumentiert läuft hinter...,früher mails dokumentiert läuft hintergrund an...,früher mails dokumentiert läuft hintergrund up...


In [14]:
# Remove numbers
def remove_num(text):

    return " ".join([word for word in str(text).split() if not word.isdigit()])


df["text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num"] = df[
    "text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare"].apply(lambda text: remove_num(text))

In [15]:
# Remove words that start with numbers e.g. 17th
def remove_num_beginn(text):
    return " ".join([word.lstrip(digits) for word in str(text).split()])


df["text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num_wo_numbeginn"] = df[
    "text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num"].apply(lambda text: remove_num_beginn(text))

In [16]:
# set seed
DetectorFactory.seed = 0

# hold label - language
languages = []

# go through each text
for ii in tqdm(range(0, len(df))):
    # split by space into list, take the first x intex, join with space
    text = df.iloc[ii]['Text'].split(" ")

    try:
        if len(text) > 50:
            lang = detect(" ".join(text[:50]))
        elif len(text) > 0:
            lang = detect(" ".join(text[:len(text)]))

    except Exception as e:
        all_words = set(text)
        try:
            lang = detect(" ".join(all_words))

        except Exception as e:
            lang = "unknown"
            pass

    # get the language
    languages.append(lang)

100%|█████████████████████████████████████████████████████████████████████████████| 6335/6335 [00:42<00:00, 150.52it/s]


In [17]:
df['language'] = languages

# Samples classified as German and English
df_De = df[df['language'] == 'de']
df_En = df[df['language'] == 'en']

In [18]:
# Samples classified as non-German and non-English
df_Other = df[(df['language'] != 'en') & (df['language'] != 'de')]

In [19]:
df_Other

,Nachrichtentyp,Datun,Uhrzeit,Absender,Ort,Text,ID,text_wo_seinfo,text_wo_seinfo_wo_enter,text_wo_seinfo_wo_enter_lower,text_wo_seinfo_wo_enter_lower_wo_punct,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stop,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num_wo_numbeginn,language
15,Beschreibung,2015.12.11,10:52:47,H2851MB7,<LOCATION>,test -- nicht beantworten!,2000000110,test -- nicht beantworten!,test -- nicht beantworten!,test -- nicht beantworten!,test nicht beantworten,test beantworten,test beantworten,test beantworten,test beantworten,test beantworten,nl
18,Beschreibung,2015.12.15,23:07:35,MASTER-ADM,<LOCATION>,test\ntest,2000000122,test\ntest,test test,test test,test test,test test,test test,test test,test test,test test,fr
24,Beschreibung,2015.12.16,17:51:23,H2851RM4,<LOCATION>,SupSup\nSup,2000000134,SupSup\nSup,SupSup Sup,supsup sup,supsup sup,supsup sup,supsup sup,,,,id
25,Beschreibung,2015.12.16,21:40:16,H2851RM4,<LOCATION>,Hello from ERPsim on HANA (P09)!\nkiss kiss,2000000135,Hello from ERPsim on HANA (P09)!\nkiss kiss,Hello from ERPsim on HANA (P09)! kiss kiss,hello from erpsim on hana (p09)! kiss kiss,hello from erpsim on hana p09 kiss kiss,hello erpsim hana p09 kiss kiss,erpsim hana p09 kiss kiss,erpsim hana p09 kiss kiss,erpsim hana p09 kiss kiss,erpsim hana p09 kiss kiss,fi
29,Beschreibung,2015.12.18,01:23:09,H2851RM1,<LOCATION>,Help!,2000000150,Help!,Help!,help!,help,help,,,,,nl
32,Beschreibung,2015.12.18,12:25:08,H2851RM1,<LOCATION>,Afhhk,2000000153,Afhhk,Afhhk,afhhk,afhhk,afhhk,afhhk,,,,sq
39,Beschreibung,2015.12.23,19:25:50,H2851LP1,<LOCATION>,TEST,2000000183,TEST,TEST,test,test,test,test,test,test,test,vi
94,Beschreibung,2016.01.21,17:13:40,H2851LP1,<LOCATION>,Test,2000000276,Test,Test,test,test,test,test,test,test,test,et
95,Beschreibung,2016.01.21,17:15:13,H2851LP1,<LOCATION>,Test,2000000277,Test,Test,test,test,test,test,test,test,test,et
134,Beschreibung,2016.01.29,14:13:28,H2851MB6,<LOCATION>,test,2000000316,test,test,test,test,test,test,test,test,test,et


In [20]:
# stemming for english
stemmer_en = SnowballStemmer("english")
def stem_words_en(text):
    return " ".join([stemmer_en.stem(word) for word in text.split()])

df_En["text_stemmed"] = df_En["text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num_wo_numbeginn"].apply(lambda text: stem_words_en(text))
df_En.head()


,Nachrichtentyp,Datun,Uhrzeit,Absender,Ort,Text,ID,text_wo_seinfo,text_wo_seinfo_wo_enter,text_wo_seinfo_wo_enter_lower,text_wo_seinfo_wo_enter_lower_wo_punct,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stop,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num_wo_numbeginn,language,text_stemmed
6,Beschreibung,2015.12.03,15:03:07,U2GB007PA1,<LOCATION>,"One of our Instructors, <PERSON16> with U-User...",2000000090,"One of our Instructors, with U-User ID U2GB00...","One of our Instructors, with U-User ID U2GB00...","one of our instructors, with u-user id u2gb00...",one of our instructors with u user id u2gb00...,one instructors u user id u2gb007am2 logged sa...,instructors u user id u2gb007am2 logged emea p...,instructors u user id u2gb007am2 logged emea p...,instructors u user id u2gb007am2 logged emea p...,instructors u user id u2gb007am2 logged emea p...,en,instructor u user id u2gb007am2 log emea porta...
10,Beschreibung,2015.12.03,18:37:16,U2GB007PA1,<LOCATION>,"One of our Instructors, <PERSON16> with U-User...",2000000094,"One of our Instructors, with U-User ID U2GB00...","One of our Instructors, with U-User ID U2GB00...","one of our instructors, with u-user id u2gb00...",one of our instructors with u user id u2gb00...,one instructors u user id u2gb007am2 longer lo...,instructors u user id u2gb007am2 longer login ...,instructors u user id u2gb007am2 longer login ...,instructors u user id u2gb007am2 longer login ...,instructors u user id u2gb007am2 longer login ...,en,instructor u user id u2gb007am2 longer login e...
16,Beschreibung,2015.12.15,15:04:14,U2GB007PA1,<LOCATION>,This is in relation to my ongoing efforts to g...,2000000120,This is in relation to my ongoing efforts to g...,This is in relation to my ongoing efforts to g...,this is in relation to my ongoing efforts to g...,this is in relation to my ongoing efforts to g...,relation ongoing efforts get instructor able u...,relation ongoing efforts instructor able servi...,relation instructor able service desk via emea...,relation instructor able service desk via emea...,relation instructor able service desk via emea...,en,relat instructor abl servic desk via emea port...
17,Beschreibung,2015.12.15,15:19:39,U2GB007PA1,<LOCATION>,I would like instructor <PERSON23> who is regi...,2000000121,I would like instructor who is registered on ...,I would like instructor who is registered on ...,i would like instructor who is registered on ...,i would like instructor who is registered on ...,would like instructor registered courses able ...,instructor registered courses able access serv...,instructor registered courses able access serv...,instructor registered courses able access serv...,instructor registered courses able access serv...,en,instructor regist cours abl access servic desk...
19,Beschreibung,2015.12.15,23:33:59,SMB_I4000001,<LOCATION>,Hello from SAPGui on system I40\n(<LOCATION18>...,2000000123,Hello from SAPGui on system I40\n()~Pimpgang!~,Hello from SAPGui on system I40 ()~Pimpgang!~,hello from sapgui on system i40 ()~pimpgang!~,hello from sapgui on system i40 pimpgang,hello sapgui system i40 pimpgang,sapgui i40 pimpgang,sapgui i40,sapgui i40,sapgui i40,en,sapgui i40


In [21]:
# stemming for german
stemmer_de = SnowballStemmer("german")
def stem_words_de(text):
    return " ".join([stemmer_de.stem(word) for word in text.split()])

df_De["text_stemmed"] = df_De["text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num_wo_numbeginn"].apply(lambda text: stem_words_de(text))
df_De.head()

,Nachrichtentyp,Datun,Uhrzeit,Absender,Ort,Text,ID,text_wo_seinfo,text_wo_seinfo_wo_enter,text_wo_seinfo_wo_enter_lower,text_wo_seinfo_wo_enter_lower_wo_punct,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stop,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num_wo_numbeginn,language,text_stemmed
0,Beschreibung,2015.11.25,19:35:19,H2451TK1,<LOCATION>,Hallo zusammen\n\naktuell sind die Systeme i06...,2000000060,Hallo zusammen\n\naktuell sind die Systeme i06...,"Hallo zusammen aktuell sind die Systeme i06, i...","hallo zusammen aktuell sind die systeme i06, i...",hallo zusammen aktuell sind die systeme i06 i...,hallo zusammen aktuell systeme i06 i58 i72 err...,aktuell systeme i06 i58 i72 erreichbar fehlerm...,aktuell systeme i06 i58 i72 erreichbar fehlerm...,aktuell systeme i06 i58 i72 erreichbar fehlerm...,aktuell systeme i06 i58 i72 erreichbar fehlerm...,de,aktuell system i06 i58 i72 erreichbar fehlerme...
1,Beschreibung,2015.11.26,14:49:28,H2741CR1,<LOCATION>,"Ich weiß nicht, woher ich die SW Information D...",2000000070,"Ich weiß nicht, woher ich die SW Information D...","Ich weiß nicht, woher ich die SW Information D...","ich weiß nicht, woher ich die sw information d...",ich weiß nicht woher ich die sw information d...,weiß woher sw information design tool bekomme ...,weiß woher sw design tool bekomme daher auskun...,weiß woher sw design tool bekomme daher auskun...,weiß woher sw design tool bekomme daher auskun...,weiß woher sw design tool bekomme daher auskun...,de,weiss woh sw design tool bekomm dah auskunft n...
2,Beschreibung,2015.11.26,15:37:47,H2741CR1,<LOCATION>,Da ich keine neue Connection anlegen kann (ist...,2000000071,Da ich keine neue Connection anlegen kann (ist...,Da ich keine neue Connection anlegen kann (ist...,da ich keine neue connection anlegen kann (ist...,da ich keine neue connection anlegen kann ist...,neue connection anlegen ausgegraut nehme beste...,neue connection anlegen ausgegraut nehme beste...,neue connection anlegen ausgegraut nehme beste...,neue connection anlegen ausgegraut nehme beste...,neue connection anlegen ausgegraut nehme beste...,de,neu connection anleg ausgegraut nehm besteh co...
3,Beschreibung,2015.11.30,11:14:57,H2702AR1,<LOCATION>,"Guten Morgen, SAP UCC Team,\n\nich würde gerne...",2000000080,"Guten Morgen, SAP UCC Team,\n\nich würde gerne...","Guten Morgen, SAP UCC Team, ich würde gerne di...","guten morgen, sap ucc team, ich würde gerne di...",guten morgen sap ucc team ich würde gerne di...,guten morgen sap ucc team gerne kursmaterialie...,guten morgen team kursmaterialien erp configur...,guten morgen team erp configuration gbi nutzen...,guten morgen team erp configuration gbi nutzen...,guten morgen team erp configuration gbi nutzen...,de,gut morg team erp configuration gbi nutz i77 m...
4,Beschreibung,2015.12.01,10:06:57,H2941JK1,<LOCATION>,"Wie bereits früher in Mails dokumentiert, läuf...",2000000081,"Wie bereits früher in Mails dokumentiert, läuf...","Wie bereits früher in Mails dokumentiert, läuf...","wie bereits früher in mails dokumentiert, läuf...",wie bereits früher in mails dokumentiert läuf...,bereits früher mails dokumentiert läuft hinter...,früher mails dokumentiert läuft hintergrund an...,früher mails dokumentiert läuft hintergrund up...,früher mails dokumentiert läuft hintergrund up...,früher mails dokumentiert läuft hintergrund up...,de,fruh mail dokumentiert lauft hintergrund updat...


In [22]:
# lemmatization for english
def lemmatizer(text):
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)


nlp = spacy.load('en_core_web_sm')

df_En["text_lemmatization"] = df_En["text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num_wo_numbeginn"].apply(
    lambda text: lemmatizer(text))
df_En.head()

,Nachrichtentyp,Datun,Uhrzeit,Absender,Ort,Text,ID,text_wo_seinfo,text_wo_seinfo_wo_enter,text_wo_seinfo_wo_enter_lower,text_wo_seinfo_wo_enter_lower_wo_punct,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stop,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num_wo_numbeginn,language,text_stemmed,text_lemmatization
6,Beschreibung,2015.12.03,15:03:07,U2GB007PA1,<LOCATION>,"One of our Instructors, <PERSON16> with U-User...",2000000090,"One of our Instructors, with U-User ID U2GB00...","One of our Instructors, with U-User ID U2GB00...","one of our instructors, with u-user id u2gb00...",one of our instructors with u user id u2gb00...,one instructors u user id u2gb007am2 logged sa...,instructors u user id u2gb007am2 logged emea p...,instructors u user id u2gb007am2 logged emea p...,instructors u user id u2gb007am2 logged emea p...,instructors u user id u2gb007am2 logged emea p...,en,instructor u user id u2gb007am2 log emea porta...,instructor u user -PRON- would u2gb007am2 log ...
10,Beschreibung,2015.12.03,18:37:16,U2GB007PA1,<LOCATION>,"One of our Instructors, <PERSON16> with U-User...",2000000094,"One of our Instructors, with U-User ID U2GB00...","One of our Instructors, with U-User ID U2GB00...","one of our instructors, with u-user id u2gb00...",one of our instructors with u user id u2gb00...,one instructors u user id u2gb007am2 longer lo...,instructors u user id u2gb007am2 longer login ...,instructors u user id u2gb007am2 longer login ...,instructors u user id u2gb007am2 longer login ...,instructors u user id u2gb007am2 longer login ...,en,instructor u user id u2gb007am2 longer login e...,instructor u user -PRON- would u2gb007am2 long...
16,Beschreibung,2015.12.15,15:04:14,U2GB007PA1,<LOCATION>,This is in relation to my ongoing efforts to g...,2000000120,This is in relation to my ongoing efforts to g...,This is in relation to my ongoing efforts to g...,this is in relation to my ongoing efforts to g...,this is in relation to my ongoing efforts to g...,relation ongoing efforts get instructor able u...,relation ongoing efforts instructor able servi...,relation instructor able service desk via emea...,relation instructor able service desk via emea...,relation instructor able service desk via emea...,en,relat instructor abl servic desk via emea port...,relation instructor able service desk via emea...
17,Beschreibung,2015.12.15,15:19:39,U2GB007PA1,<LOCATION>,I would like instructor <PERSON23> who is regi...,2000000121,I would like instructor who is registered on ...,I would like instructor who is registered on ...,i would like instructor who is registered on ...,i would like instructor who is registered on ...,would like instructor registered courses able ...,instructor registered courses able access serv...,instructor registered courses able access serv...,instructor registered courses able access serv...,instructor registered courses able access serv...,en,instructor regist cours abl access servic desk...,instructor register course able access service...
19,Beschreibung,2015.12.15,23:33:59,SMB_I4000001,<LOCATION>,Hello from SAPGui on system I40\n(<LOCATION18>...,2000000123,Hello from SAPGui on system I40\n()~Pimpgang!~,Hello from SAPGui on system I40 ()~Pimpgang!~,hello from sapgui on system i40 ()~pimpgang!~,hello from sapgui on system i40 pimpgang,hello sapgui system i40 pimpgang,sapgui i40 pimpgang,sapgui i40,sapgui i40,sapgui i40,en,sapgui i40,sapgui i40


In [23]:
# lemmatization for german
nlp = spacy.load('de_core_news_sm')

df_De["text_lemmatization"] = df_De["text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num_wo_numbeginn"].apply(
    lambda text: lemmatizer(text))
df_De.head()

,Nachrichtentyp,Datun,Uhrzeit,Absender,Ort,Text,ID,text_wo_seinfo,text_wo_seinfo_wo_enter,text_wo_seinfo_wo_enter_lower,text_wo_seinfo_wo_enter_lower_wo_punct,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stop,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num,text_wo_seinfo_wo_enter_lower_wo_punct_wo_stoptext_wo_stopfreq_wo_stopfreqrare_wo_num_wo_numbeginn,language,text_stemmed,text_lemmatization
0,Beschreibung,2015.11.25,19:35:19,H2451TK1,<LOCATION>,Hallo zusammen\n\naktuell sind die Systeme i06...,2000000060,Hallo zusammen\n\naktuell sind die Systeme i06...,"Hallo zusammen aktuell sind die Systeme i06, i...","hallo zusammen aktuell sind die systeme i06, i...",hallo zusammen aktuell sind die systeme i06 i...,hallo zusammen aktuell systeme i06 i58 i72 err...,aktuell systeme i06 i58 i72 erreichbar fehlerm...,aktuell systeme i06 i58 i72 erreichbar fehlerm...,aktuell systeme i06 i58 i72 erreichbar fehlerm...,aktuell systeme i06 i58 i72 erreichbar fehlerm...,de,aktuell system i06 i58 i72 erreichbar fehlerme...,aktuell systeme i06 i58 i72 erreichbar fehlerm...
1,Beschreibung,2015.11.26,14:49:28,H2741CR1,<LOCATION>,"Ich weiß nicht, woher ich die SW Information D...",2000000070,"Ich weiß nicht, woher ich die SW Information D...","Ich weiß nicht, woher ich die SW Information D...","ich weiß nicht, woher ich die sw information d...",ich weiß nicht woher ich die sw information d...,weiß woher sw information design tool bekomme ...,weiß woher sw design tool bekomme daher auskun...,weiß woher sw design tool bekomme daher auskun...,weiß woher sw design tool bekomme daher auskun...,weiß woher sw design tool bekomme daher auskun...,de,weiss woh sw design tool bekomm dah auskunft n...,weiß woher sw design tool bekommen daher ausku...
2,Beschreibung,2015.11.26,15:37:47,H2741CR1,<LOCATION>,Da ich keine neue Connection anlegen kann (ist...,2000000071,Da ich keine neue Connection anlegen kann (ist...,Da ich keine neue Connection anlegen kann (ist...,da ich keine neue connection anlegen kann (ist...,da ich keine neue connection anlegen kann ist...,neue connection anlegen ausgegraut nehme beste...,neue connection anlegen ausgegraut nehme beste...,neue connection anlegen ausgegraut nehme beste...,neue connection anlegen ausgegraut nehme beste...,neue connection anlegen ausgegraut nehme beste...,de,neu connection anleg ausgegraut nehm besteh co...,neue connection anlegen ausgegraut nehmen best...
3,Beschreibung,2015.11.30,11:14:57,H2702AR1,<LOCATION>,"Guten Morgen, SAP UCC Team,\n\nich würde gerne...",2000000080,"Guten Morgen, SAP UCC Team,\n\nich würde gerne...","Guten Morgen, SAP UCC Team, ich würde gerne di...","guten morgen, sap ucc team, ich würde gerne di...",guten morgen sap ucc team ich würde gerne di...,guten morgen sap ucc team gerne kursmaterialie...,guten morgen team kursmaterialien erp configur...,guten morgen team erp configuration gbi nutzen...,guten morgen team erp configuration gbi nutzen...,guten morgen team erp configuration gbi nutzen...,de,gut morg team erp configuration gbi nutz i77 m...,gut morgen team erp configuration gbi nutzen i...
4,Beschreibung,2015.12.01,10:06:57,H2941JK1,<LOCATION>,"Wie bereits früher in Mails dokumentiert, läuf...",2000000081,"Wie bereits früher in Mails dokumentiert, läuf...","Wie bereits früher in Mails dokumentiert, läuf...","wie bereits früher in mails dokumentiert, läuf...",wie bereits früher in mails dokumentiert läuf...,bereits früher mails dokumentiert läuft hinter...,früher mails dokumentiert läuft hintergrund an...,früher mails dokumentiert läuft hintergrund up...,früher mails dokumentiert läuft hintergrund up...,früher mails dokumentiert läuft hintergrund up...,de,fruh mail dokumentiert lauft hintergrund updat...,früh mails dokumentieren laufen hintergrund up...


# Save the preprocessed data

In [24]:
df_En.to_csv("preprocessed_data_en.csv",index=False)

In [25]:
df_De.to_csv("preprocessed_data_de.csv",index=False)